In [1]:
import numpy as np
import pandas as pd
from datetime import datetime
from dateutil.relativedelta import relativedelta
from dateparser import parse
from copy import deepcopy

In [2]:
import os
os.chdir("../")

In [3]:
from pandemic_functions.delphi_functions.DELPHI_model_policy_scenarios import read_oxford_country_policy_data, read_policy_data_us_only, get_dominant_policy
from pandemic_functions.pandemic_params import future_policies

/Users/saksham/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3263: DtypeWarning: Columns (2,3) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


In [4]:
start_date = "2020-03-01"
end_date =  "2021-01-31"

country = 'US'
province = 'Florida'

if country == 'US':
    policy_data = read_policy_data_us_only(province, start_date=start_date, end_date=end_date)
else:
    policy_data = read_oxford_country_policy_data(country=country, start_date=start_date, end_date=end_date)

In [27]:
end_dt = parse(end_date)

dt1 = parse(start_date)
dt2 = dt1 + relativedelta(months=1, days=-1)

dominant_policy_dict = {"month_of": [], "dominant_policy": []}

while dt2 <= end_dt:
    dominant_policy_dict["month_of"].append(dt1.date())
    dominant_policy_dict["dominant_policy"].append(get_dominant_policy(policy_data, start_date=dt1, end_date=dt2))
    dt1 = dt2 + relativedelta(days=1)
    dt2 = dt1 + relativedelta(months=1, days=-1)

dominant_policy_df = pd.DataFrame.from_dict(dominant_policy_dict)

In [28]:
dominant_policy_df

,month_of,dominant_policy
0,2020-03-01,No_Measure
1,2020-04-01,Lockdown
2,2020-05-01,Lockdown
3,2020-06-01,Mass_Gatherings_Authorized_But_Others_Restricted
4,2020-07-01,Mass_Gatherings_Authorized_But_Others_Restricted
5,2020-08-01,Mass_Gatherings_Authorized_But_Others_Restricted
6,2020-09-01,Mass_Gatherings_Authorized_But_Others_Restricted
7,2020-10-01,Mass_Gatherings_Authorized_But_Others_Restricted
8,2020-11-01,Mass_Gatherings_Authorized_But_Others_Restricted
9,2020-12-01,Mass_Gatherings_Authorized_But_Others_Restricted
